In [1]:
%%capture
!pip install JAEN -qU

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchinfo import summary
from JAEN.utils import plot_training_results

# device 설정 (GPU가 사용 가능하면 GPU로, 그렇지 않으면 CPU 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
import os

# JAEN 모듈 불러오기
from JAEN.datasets import load_titanic

# 데이터 파일이 저장된 경로 찾기
jaen_path = os.path.dirname(load_titanic.__code__.co_filename)
train_path = os.path.join(jaen_path, "data", "00", "train_loader.pt")
test_path = os.path.join(jaen_path, "data", "00", "test_loader.pt")

# torch.load()를 직접 실행 (weights_only=False 설정)
train_loader = torch.load(train_path, weights_only=False)
test_loader = torch.load(test_path, weights_only=False)

# 데이터 확인
print(train_loader)
print(test_loader)

In [ ]:
model = nn.Sequential(
    nn.Linear(7, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)

# 모델 인스턴스 생성
model = model.to(device)
summary(model, (32, 7))

# 32 → 한 번에 모델에 입력하는 샘플 개수 (Batch Size)
# 7 → 각 샘플의 특성(feature) 개수

## nn.Module 기반 신경망 모델 구성

In [5]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7, 32)
        self.fc2 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# 모델 인스턴스 생성
model = DNN().to(device)
summary(model, (32, 7))

Layer (type:depth-idx)                   Output Shape              Param #
DNN                                      [32, 1]                   --
├─Linear: 1-1                            [32, 32]                  256
├─ReLU: 1-2                              [32, 32]                  --
├─Linear: 1-3                            [32, 1]                   33
├─Sigmoid: 1-4                           [32, 1]                   --
Total params: 289
Trainable params: 289
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

## 모델 학습 코드

In [6]:
criterion = nn.BCELoss()  # 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.01)  # 옵티마이저

#### 모델 학습 함수 구현

In [7]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # 모델을 학습 모드로 설정

    running_loss = 0.0 # 미니 배치별 loss값을 누적할 변수

    for data, labels in train_loader: # 미니 배치 별 파라미터 업데이트 수행
        data, labels = data.to(device), labels.to(device) # 미니 배치별 데이터와 레이블 장치 할당

        # 순전파
        outputs = model(data)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 기울기 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 파라미터 업데이트
        optimizer.step()

        # 손실 누적
        running_loss += loss.item()

    # 현재 Epoch의 평균 손실 값 계산 및 반환
    return running_loss / len(train_loader)

#### 모델 평가 함수 구현

In [8]:
# 평가 함수 정의
def evaluate(model, test_loader, criterion, device):
    model.eval()  # 모델을 평가 모드로 설정

    running_loss = 0.0 # 미니 배치별 loss값을 누적할 변수

    with torch.no_grad():  # 평가 중에는 기울기 계산을 하지 않음
        for data, labels in test_loader: # 미니 배치 별 손실 계산
            data, labels = data.to(device), labels.to(device) # 미니 배치별 데이터와 레이블 장치 할당

            # 순전파
            outputs = model(data)

            # 손실 계산
            loss = criterion(outputs, labels)

            # 손실 누적
            running_loss += loss.item()


    # 현재 Epoch의 평균 손실 값 계산 및 반환
    return running_loss / len(test_loader)

#### 학습 및 평가

In [10]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(16)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

model = DNN().to(device)
summary(model, input_size = (32, 7))

Layer (type:depth-idx)                   Output Shape              Param #
DNN                                      [32, 1]                   --
├─Linear: 1-1                            [32, 64]                  512
├─BatchNorm1d: 1-2                       [32, 64]                  128
├─ReLU: 1-3                              [32, 64]                  --
├─Dropout: 1-4                           [32, 64]                  --
├─Linear: 1-5                            [32, 32]                  2,080
├─BatchNorm1d: 1-6                       [32, 32]                  64
├─ReLU: 1-7                              [32, 32]                  --
├─Linear: 1-8                            [32, 16]                  528
├─BatchNorm1d: 1-9                       [32, 16]                  32
├─ReLU: 1-10                             [32, 16]                  --
├─Linear: 1-11                           [32, 1]                   17
├─Sigmoid: 1-12                          [32, 1]                   --
Total par

In [11]:
train_losses = []
test_losses = []

# 학습 횟수 만큼 반복
for epoch in range(100):

    # 모델 학습(학습데이터)
    train_loss = train(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    # 모델 평가 (평가데이터)
    test_loss = evaluate(model, test_loader, criterion, device)
    test_losses.append(test_loss)

    print(f'Epoch {epoch+1} Train Loss : {train_loss} Test Loss : {test_loss}')

Epoch 1 Train Loss : 0.6935758279717487 Test Loss : 0.6815032462279002
Epoch 2 Train Loss : 0.6974595059519229 Test Loss : 0.6970425049463908
Epoch 3 Train Loss : 0.6964311107345249 Test Loss : 0.6977594395478567
Epoch 4 Train Loss : 0.7055266940075419 Test Loss : 0.6992875138918558
Epoch 5 Train Loss : 0.6923974285955015 Test Loss : 0.6998989482720693
Epoch 6 Train Loss : 0.7016270497570867 Test Loss : 0.6952687601248423
Epoch 7 Train Loss : 0.695244400397591 Test Loss : 0.7043977777163187
Epoch 8 Train Loss : 0.7001471778620845 Test Loss : 0.7060245970884959
Epoch 9 Train Loss : 0.6988425228906714 Test Loss : 0.7010207573572794
Epoch 10 Train Loss : 0.6998503804206848 Test Loss : 0.7060646017392477
Epoch 11 Train Loss : 0.6945584442304529 Test Loss : 0.7065034906069437
Epoch 12 Train Loss : 0.6967361569404602 Test Loss : 0.7019996146361033
Epoch 13 Train Loss : 0.6895307483880416 Test Loss : 0.7011136809984843
Epoch 14 Train Loss : 0.6951934928479402 Test Loss : 0.6976640721162161
Ep

In [ ]:
from JAEN.utils import plot_training_results
plot_training_results(train_losses, test_losses)

#### 드랍아웃을 포함한 모델 클래스 구현

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7, 32)
        self.fc2 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.1)  # 10%의 드랍아웃 적용
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# 모델 인스턴스화
model = DNN().to(device)
summary(model, (32, 7))

#### Batch Normalization 및 Droupout 포함 모델 정의

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(16)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

model = DNN().to(device)
summary(model, input_size=(32, 7))